In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string
from glob import glob
from sklearn.utils import shuffle
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

%matplotlib inline

slim = tf.contrib.slim
print(tf.__version__)


/Users/sw/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/sw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.1


## 데이터 불러오기

In [6]:
df = pd.read_csv('./dataset/josa_none_dataset.csv', sep=',')
new_df = df.dropna()
# new_df["reivew_split"] = new_df["review"].str.split()

In [7]:
new_df = new_df[:32000]

In [8]:
new_df

,review,rate,emotion,review2
0,원작이 칭송받는 이유는 웹툰 계 자체의 질적 저하가 심각하기 때문. 원작이나 영화...,1,0,원작 칭송 받다 이유 웹툰 계 자체 질적 저하 심각하다 때문 원작 영화 별로 건 마찬가지
1,나름의 감동도 있고 안타까운 마음에 가슴도 먹먹 배우들의 연기가 good 김수현...,10,1,나름 감동 있다 안타깝다 마음 가슴 먹다 먹다 배우 들 연기 good 김수현 최고
2,이런걸 돈주고 본 내자신이 후회스럽다 최악의 쓰레기 영화 김수현 밖에없는 저질 삼류영화,1,0,이런 걸 돈 주다 보다 내 자신 후회 스럽다 최악 쓰레기 영화 김수현 밖 없다 저 ...
3,"초반엔 코미디, 후반엔 액션, 결론은 코미디.",7,1,초반 코미디 후반 액션 결론 코미디
4,원작 어디에 팔아 먹었죠????,1,0,원작 어디 팔 먹다
5,님들이영화만들어보세요; 하는 분들 묶어두고 클레멘타인보여줘야한다.,1,0,님 들 이영화 만들다 보다 하다 분들 묶다 두다 클레멘타인 보여주다 야하다
6,재밌긴한데 좀 억지감동짜냄 좀 유치함,7,1,재밌다 좀 억 지 감동 짜내다 좀 유치하다
7,이게 어떻게 흥행했는지 궁금하다,1,0,이 어떻다 흥행 하다 궁금하다
8,내용이 정말 감동적이였다,9,1,내용 정말 감동 적다
9,김수현 연기 굿!!! ^^,10,1,김수현 연기 굿


In [5]:
emotion_class = [
    "POSI" if new_df.iloc[i]['rate'] >= 8
    else
    "SOSO" if new_df.iloc[i]['rate'] >= 4
    else
    "NEGA"
    for i in range(new_df.shape[0])
]
new_df["class"] = emotion_class

In [9]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(new_df["review2"])

if num_words is None:
    num_words = len(tokenizer.word_index)

In [10]:
x_tokens = tokenizer.texts_to_sequences(new_df["review2"])

x_tokens[0]

num_tokens = [len(tokens) for tokens in x_tokens]
num_tokens = np.array(num_tokens)

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

np.sum(num_tokens < max_tokens) / len(num_tokens)

pad = 'pre'

x_tokens_pad = pad_sequences(x_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_tokens_pad.shape

x_tokens_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,  269,  111,  331, 1607, 1043,  220, 4496, 1477,  142,  269,
          1,  203,  342, 1943], dtype=int32)

In [11]:
X_test_np = x_tokens_pad[:int(len(new_df)*0.8)]
X_train_np = x_tokens_pad[int(len(new_df)*0.8):]

n_values = np.max(new_df["emotion"]) + 1
y = np.eye(n_values)[new_df["emotion"]]

y_test_np = y[:int(len(new_df)*0.8)]
y_train_np = y[int(len(new_df)*0.8):]

In [12]:
len(y_train_np)

6400

## doc2vec

In [10]:
# from gensim.models.doc2vec import Doc2Vec
# from collections import namedtuple

In [11]:
# train_df = new_df[:int(len(new_df)*0.8)]
# test_df = new_df[int(len(new_df)*0.8):]

# TaggedDocument = namedtuple('TaggedDocument', 'words tags')
# tagged_train_docs = [TaggedDocument(d, c) for d, c in train_df[['reivew_split', 'class']].values]
# tagged_test_docs = [TaggedDocument(d, c) for d, c in test_df[['reivew_split', 'class']].values]


In [12]:
# import multiprocessing
# cores = multiprocessing.cpu_count()

In [13]:
# doc_vectorizer = Doc2Vec(
#     dm=0,            # PV-DBOW / default 1
#     dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
#     window=8,        # distance between the predicted word and context words
#     size=300,        # vector size
#     alpha=0.025,     # learning-rate
#     seed=1234,
#     min_count=20,    # ignore with freq lower
#     min_alpha=0.025, # min learning-rate
#     workers=cores,   # multi cpu
#     hs = 1,          # hierarchical softmax / default 0
#     negative = 10,
# )

In [14]:
# doc_vectorizer.build_vocab(tagged_train_docs)

In [15]:
# doc_vectorizer.corpus_count
# doc_vectorizer.iter

In [16]:
# for epoch in range(10):
#     doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
#     doc_vectorizer.alpha -= 0.002 # decrease the learning rate
#     doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

In [17]:
# model_name = 'Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
# doc_vectorizer.save(model_name)

In [18]:
# Load model
# doc_vectorizer = Doc2Vec.load(model_name)

In [19]:
# X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
# y_train = [doc.tags for doc in tagged_train_docs]

# X_test = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
# y_test = [doc.tags for doc in tagged_test_docs]

## 파라미터 설정

In [13]:
# Training Parameters
learning_rate = 0.01
training_steps = 100000
batch_size = 100
display_step = 100

# Network Parameters
num_input = 17 # MNIST data input (img shape: 28*28)
timesteps = 2 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 3 # MNIST total classes (0-9 digits)

## 데이터셋 설정

In [21]:
# X_train_np = np.asarray(X_train)
# y_train_np = np.asarray([0 if c == 'NEGA' else 1 if c == 'SOSO' else 2 for c in y_train], dtype=int)

# X_test_np = np.asarray(X_test)
# y_test_np = np.asarray([0 if c == 'NEGA' else 1 if c == 'SOSO' else 2 for c in y_test], dtype=int)

In [22]:
# X_test_np

In [23]:
# y_train_np = np.eye(3)[y_train_np.reshape(-1)]
# y_test_np = np.eye(3)[y_test_np.reshape(-1)]

In [24]:
# y_train_np[0]

In [14]:
from tensorflow.contrib import rnn

## placeholder 설정

In [15]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [16]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [17]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [18]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        for i in range(0, len(X_train_np), batch_size):
            batch_xs = X_train_np[i:i+batch_size]
            batch_ys = y_train_np[i:i+batch_size]
            batch_xs = batch_xs.reshape((batch_size, timesteps, num_input))
            sess.run(train_op, feed_dict={X: batch_xs, Y: batch_ys})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_xs,
                                                                 Y: batch_ys})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            
    print("Optimization Finished!")

#     Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = X_test_np[:test_len].reshape((-1, timesteps, num_input))
    test_label = y_test_np[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
    

ValueError: cannot reshape array of size 2600 into shape (100,2,17)